In [1]:
from transformers import AutoTokenizer, DataCollatorWithPadding,TFAutoModelForSequenceClassification
from datasets import load_dataset
import tensorflow as tf

/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-01-14 14:45:12.930850: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-14 14:45:12.930991: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-14 14:45:13.264898: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-14 14:45:13.961353: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow bi

In [11]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json: 100%|██████████| 28.0/28.0 [00:00<00:00, 145kB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 473kB/s]
tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 634kB/s]


In [12]:
reviews = load_dataset("csv", data_files="movie_reviews.csv")
reviews["train"] = load_dataset("csv", data_files="movie_reviews.csv", split='train[:1%]')
reviews

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 400
    })
})

In [13]:
def preprocess_function(batch):
    return tokenizer(batch["text"], padding=True, truncation=True, return_tensors="tf")

In [14]:
tokenized_reviews = reviews.map(preprocess_function, batched=True)

Map: 100%|██████████| 400/400 [00:00<00:00, 1051.08 examples/s]


In [15]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [16]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}
model = TFAutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
from transformers import create_optimizer

batch_size = 16
num_epochs = 5
batches_per_epoch = len(tokenized_reviews["train"])
total_train_steps = int(batches_per_epoch * num_epochs)

In [18]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_reviews["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [19]:
from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(3e-5)) 

In [20]:
import numpy as np
import evaluate

accuracy = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [23]:
model.fit(tf_train_set)

Epoch 1/3


: 